### Environment Preparation 

* prepare session and bucket 
* prepare necessary parameters 

In [1]:
import sagemaker 
import boto3 
sess = sagemaker.Session()
BUCKET = sess.default_bucket()

In [2]:
BUCKET

'sagemaker-us-west-2-400776389993'

In [3]:
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]


In [4]:
!ls 


api.sh	content_image  prepare-lambda-functions.ipynb  process_a2i_results


### Setup lambda functions and SQS 
We use a lambda function ```invoke_image_object_detection``` to 
1. trigger the sagemaker endpoint to do inference
2. save the image to s3 and trigger a2i process 
3. send a2i and s3 information to SQS 

And, we use a lambda function ```process_a2i_results``` to 
1. listening to SQS and process a2i results 
2. trigger retraining pipelines 

The following steps creates lambda functions 

* copying codes to s3 
* create lambda functions by codes on s3 
* setup environment variables 


In [5]:
%%bash -s "$BUCKET" 
cd ./process_a2i_results
zip -r process_a2i_results.zip  .
aws s3 cp process_a2i_results.zip s3://$1/lambda/

updating: model.py (deflated 47%)
updating: lambda_function.py (deflated 65%)
updating: prepare_data.py (deflated 43%)
upload: ./process_a2i_results.zip to s3://sagemaker-us-west-2-400776389993/lambda/process_a2i_results.zip


In [6]:
%store -r role 

In [7]:
role

'arn:aws:iam::400776389993:role/SageMakerExecutionRole'

In [8]:
import os
cwd = os.getcwd()
!aws lambda create-function --function-name process_a2i_results --zip-file fileb://$cwd/process_a2i_results/process_a2i_results.zip  --handler lambda_function.lambda_handler --runtime python3.7 --role $role


An error occurred (ResourceConflictException) when calling the CreateFunction operation: Function already exist: process_a2i_results


#### Configure lambda function - process_a2i_results
![configure environment variable](./content_image/setup_env_vars_for_lambda3.png)

In [9]:
%store -r model_package_group_name 
%store -r pipeline_name

In [10]:
bucket_key = "a2i-demo"
variables = f"MODEL_GROUP={model_package_group_name},BUCKET={BUCKET},PIPELINE={pipeline_name}"
env = "Variables={"+variables+"}"

!aws lambda update-function-configuration --function-name process_a2i_results --environment "$env"

{
    "FunctionName": "process_a2i_results",
    "FunctionArn": "arn:aws:lambda:us-west-2:400776389993:function:process_a2i_results",
    "Runtime": "python3.7",
    "Role": "arn:aws:iam::400776389993:role/SageMakerExecutionRole",
    "Handler": "lambda_function.lambda_handler",
    "CodeSize": 2282,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2021-06-01T05:13:43.844+0000",
    "CodeSha256": "2iWClprr5B1scIxcMStESwgO/97de5OpSm4EYKvm88w=",
    "Version": "$LATEST",
    "Environment": {
        "Variables": {
            "BUCKET": "sagemaker-us-west-2-400776389993",
            "PIPELINE": "AudioClassification",
            "MODEL_GROUP": "AudioClassificationGroupModel"
        }
    },
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "6e2c8169-0ef1-449a-8731-6d9513cd110c",
    "State": "Active",
    "LastUpdateStatus": "Successful",
    "PackageType": "Zip"
}


### Set up SQS 

* we use SQS to pass the information between lambda functions in this solution 
* set up SQS and attach it as a trigger to a lambda function 



In [11]:
!aws lambda create-event-source-mapping --function-name process_a2i_results  --batch-size 10 \
--maximum-batching-window-in-seconds 60 --event-source-arn arn:aws:sqs:us-west-2:$account_id:a2itasks



An error occurred (ResourceConflictException) when calling the CreateEventSourceMapping operation: An event source mapping with SQS arn (" arn:aws:sqs:us-west-2:400776389993:a2itasks ") and function (" process_a2i_results ") already exists. Please update or delete the existing mapping with UUID 22ffd093-cfaf-458d-98f2-dcb398fd4b02


In [ ]:
!./api.sh "https://otu4uaku1e.execute-api.us-west-2.amazonaws.com/dev/classify" 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0